In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import numpy as np

from bayes_opt import *
from plotters import *
from sample_loss import *
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from matplotlib import cm

ImportError: No module named models.dc_st_pmask

In [ ]:
p_pattern = gen_pattern(a=1.3, b=2.5, r_spacing=3)
plt.figure()
plt.imshow(p_pattern)
plt.colorbar()

np.mean(p_pattern)

In [ ]:
u = np.random.uniform(size=(256, 192))
plt.imshow(p_pattern>u)
np.mean(p_pattern>u)

In [ ]:
data, target = make_classification(n_samples=2500,
                                   n_features=45,
                                   n_informative=15,
                                   n_redundant=5)

def sample_loss(params):
    print('C = {0}, gamma = {1}'.format(params[0], params[1]))
    return cross_val_score(SVC(C=10 ** params[0], gamma=10 ** params[1], random_state=12345),
                           X=data, y=target, scoring='roc_auc', cv=3).mean()

In [ ]:
lambdas = np.linspace(1, -4, 25)
gammas = np.linspace(1, -4, 20)

# We need the cartesian combination of these two vectors
param_grid = np.array([[C, gamma] for gamma in gammas for C in lambdas])

# real_loss = [sample_loss(params) for params in param_grid]
real_loss = np.load('real_loss.npy')
# np.save('real_loss.npy', real_loss)

# The maximum is at:
param_grid[np.array(real_loss).argmax(), :]

In [ ]:
C, G = np.meshgrid(lambdas, gammas)
plt.figure()
cp = plt.contourf(C, G, np.array(real_loss).reshape(C.shape), cmap=cm.jet)
plt.colorbar(cp)
plt.title('Filled contours plot of loss function L(gamma, C)')
plt.xlabel('C')
plt.ylabel('gamma')
# plt.savefig('/Users/thomashuijskens/Personal/gp-optimisation/figures/real_loss_contour.png', bbox_inches='tight')
plt.show()

In [ ]:
bounds = np.array([[-4, 1], [-4, 1]])

xp, yp = bayesian_optimisation(n_iters=30, 
                               sample_loss=sample_loss, 
                               bounds=bounds,
                               n_pre_samples=3,
                               random_search=False)

In [ ]:
plot_iteration(lambdas, xp, yp, first_iter=3, second_param_grid=gammas, optimum=[0.375, -2.15789474])